In [ ]:
import os
from pandas import read_csv
import pandas as pd

# Create Dataframe

In [ ]:
df = pd.DataFrame(columns = ['cust_no',
                             'age_level', 
                             'exp_code',
                             'source_code',
                             '0', '1', '2', '5', '9', 'A','trans_type',
                             'c0','c1','c2','c3','c4','c5','c6','current_c',
                             'H','O','R','company_type',
                             'avg_stock','current_stock',
                             'avg_amount','current_amount',
                             'label'])

# Customer Information

In [ ]:
cust_info = 'cust_info/cust_info_1.csv'
cust_info_df = read_csv(cust_info)


# Stock Category

In [ ]:
stock_cata = 'stock_category/stock_category.csv'
stock_cata_df = read_csv(stock_cata, header=None)

In [ ]:
stock_cata_df.head()

In [ ]:
company_to_cat = {}

cata_0 = ['電子–光電','電子–其他電子','電子–半導體','電子–資訊服務','電子–通信網路','電子–電子通路','電子–電子零組件','電子–電腦及週邊設備']
cata_1 = ['化學工業','汽車工業','油電燃氣','建材營建','紡織纖維','造紙工業','塑膠工業','農業科技','電器電纜','電機機械','鋼鐵工業','橡膠工業','水泥工業','玻璃陶瓷']
cata_2 = ['金融保險','管理股票']
cata_3 = ['生技醫療','食品工業']
cata_4 = ['航運業']
cata_5 = ['文化創意','存託憑證','其他','貿易百貨','電子商務','觀光事業']

for idx, data in stock_cata_df.iterrows():
    if data[1] in cata_0:
        company_to_cat[data[0]] = 0
    elif data[1] in cata_1:
        company_to_cat[data[0]] = 1
    elif data[1] in cata_2:
        company_to_cat[data[0]] = 2
    elif data[1] in cata_3:
        company_to_cat[data[0]] = 3
    elif data[1] in cata_4:
        company_to_cat[data[0]] = 4
    elif data[1] in cata_5:
        company_to_cat[data[0]] = 5
    else:
        company_to_cat[data[0]] = 6

In [ ]:
company_data = pd.DataFrame(list(company_to_cat.items()), columns=['company', 'category'])
company_data.to_csv('company_data.csv')

In [ ]:
company_data = read_csv('company_data.csv',index_col=0)
company_data.head()

# Make Data

In [ ]:
txn = [f for f in os.listdir('txn') if not f.startswith('.')]
temp = []
for i, (cust, breach_ind, breach_rank, source_code, age_level, investment_exp_code, breach_date) in enumerate(zip(cust_info_df['cust'],
                                                                                                               cust_info_df['breach_ind'], 
                                                                                                               cust_info_df['breach_rank'],
                                                                                                               cust_info_df['source_code'],
                                                                                                               cust_info_df['age_level'],
                                                                                                               cust_info_df['investment_exp_code'],
                                                                                                              cust_info_df['breach_date'])):
    
    
    
    # pass the second breach
    if breach_ind == 1 and breach_rank == 2.0:
        continue
        
    print(f'{i}/{len(cust_info_df)}',end = '\r')    
    # init for each cust
    types = {'0':0, '1':0, '2':0, '5':0, '9':0, 'A':0}
    com_types = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
    HOR = {'H':0, 'O':0, 'R':0}
    flag = 0
    stock = 0
    current_stock = 0
    amount = 0
    current_amount = 0
    num = 0
    for t in txn:
        txn_df = read_csv('txn/'+t, header = None)
        # customer filter                                                                                                    
        c_txn_df = txn_df.loc[txn_df[1] == cust].reset_index(drop=True)
        
        
        # type of 0 1 2 5 9 A
        if c_txn_df.empty:
            continue
        commission = c_txn_df[5].value_counts()
        for type_ in types:
            if type_ in commission:
                types[type_] = types[type_] + commission[type_]
                
                
        # PAST data
        for idx, row in c_txn_df.iterrows():
            # amount = price x stock
            stock += row[7]
            amount += row[6]*row[7]
            num += 1
            
            # company cat one hot
            com = row[4]
            if com in company_to_cat:
                com_types[company_to_cat[com]] = com_types[company_to_cat[com]] + 1
            else:
                com_types[6] = com_types[6] + 1
            
            #HOR
            hor = row[3]
            if hor in HOR:
                HOR[hor] = HOR[hor] + 1
            else:
                pass
            
            
            
        # CUREENT transaction type and amount
        if breach_ind == 1:
           # breach trans filter (need to modify) 
            cb_txn_df = c_txn_df.loc[c_txn_df[0] == breach_date].reset_index(drop=True)
            
            if not cb_txn_df.empty:
                flag = 1
                trans_type = cb_txn_df.at[0,5]
                current_stock = cb_txn_df.at[0,7]
                current_amount = cb_txn_df.at[0,6]*cb_txn_df.at[0,7]
                if cb_txn_df.at[0,4] in company_to_cat:
                    current_c = company_to_cat[cb_txn_df.at[0,4]]
                else:
                    current_c = 6
                company_type = cb_txn_df.at[0,3]
                
            if flag == 0:
                trans_type = 0 
                current_amount = float('nan')
                current_c = float('nan')
                company_type = 'H'
        else:
            if not c_txn_df.empty:
                flag = 1
                trans_type = c_txn_df.at[0,5]
                current_stock = c_txn_df.at[0,7]
                current_amount = c_txn_df.at[0,6]*c_txn_df.at[0,7]
                if c_txn_df.at[0,4] in company_to_cat:
                    current_c = company_to_cat[c_txn_df.at[0,4]]
                else:
                    current_c = 6
                company_type = c_txn_df.at[0,3]
    
    # pass the cust that make less than n transaction
    if num < 3 and breach_ind == 0:
        continue
        
    if num == 0 and breach_ind == 1:
        num = 1 # avoid error 'divided by 0'
        
    avg_stock = stock/num
    avg_amount = amount/num  
    
                
    # source code to 1 2 
    if source_code == 'A':
        source_code = 1
    elif source_code == 'B':
        source_code = 2
        
    # fill data
    df.loc[i] = [cust,
                 age_level, 
                 investment_exp_code, 
                 source_code,
                 types['0'], types['1'], types['2'], types['5'], types['9'], types['A'], trans_type,
                 com_types[0],com_types[1],com_types[2],com_types[3],com_types[4],com_types[5],com_types[6],current_c,
                 HOR['H'],HOR['O'],HOR['R'],company_type,
                 avg_stock,current_stock,
                 avg_amount, current_amount,
                 breach_ind]
    
    
#3235/105770   

In [ ]:
df = df.reset_index(drop=True)
df.to_csv('data.csv')

In [ ]:
df.head(10)

# Processing miss value

In [ ]:
df = read_csv('data.csv')

In [ ]:
breach_df = df.loc[df['label'] == 1]
non_breach_df = df.loc[df['label'] == 0]

# feature analyze 

In [ ]:
his_df = pd.DataFrame()

# his_df['1'] = breach_df.c6	
his_df['0'] = non_breach_df.c6	

ax = his_df.plot.hist(bins=12, alpha=0.5)